# Output Formatters Tutorial

---

## To Get the Most Out of this Tutorial
You may read more about the `output_formatter` plugin type <br/>
on the accompanying slides [hosted at this link](https://docs.google.com/presentation/d/1wFbrP4RBdwf6Hx30adD9DN4jNZm1f0hVaHxV--encS0/edit?usp=sharing).

## Importing the GeoIPS Interfaces

Since we are walking through this tutorial as a Python script, 
we need to import every plugin that we plan to use.

We use the convention:
`{interface_type}.get_plugin({plugin_name})` 
to load the plugins at runtime .

In [1]:
import geoips
from geoips import interfaces

# Since we are reading abi data in a netCDF format, we will use this reader
abi_reader = interfaces.readers.get_plugin("abi_netcdf")
# Since we will be reading a single channel, we want to use the single_channel algorithm
single_channel_algorithm = interfaces.algorithms.get_plugin("single_channel")
# We want to use Nearest Neighbor resampling for this dataset, so we will use the interp_nearest interpolator plugin
interp_nearest = interfaces.interpolators.get_plugin("interp_nearest")
# We want to restrict our output to just the CONUS sector
conus_sector_plugin = interfaces.sectors.get_plugin("conus")

Since we load these plugins dynamically, it can take some extra work to check
their call signatures.  
We can look more closely at the reader plugin by calling:
(this only works in ipython or jupyter notebooks)

In [2]:
abi_reader?

Signature:     
abi_reader(
    fnames,
    metadata_only=False,
    chans=None,
    area_def=None,
    self_register=False,
    geolocation_cache_backend='memmap',
    cache_chunk_size=None,
    cache_data=False,
    cache_solar_angles=False,
    resource_tracker=None,
    roi=None,
)
Type:           ReadersPlugin
String form:    <geoips.interfaces.base.ReadersPlugin object at 0x7fdfb1579c10>
File:           ~/miniconda3/lib/python3.11/site-packages/geoips/interfaces/base.py
Docstring:      Standard GeoIPS xarray dictionary based ABI NetCDF data reader.
Call docstring:
Read ABI NetCDF data from a list of filenames.

Parameters
----------
fnames : list
    * List of strings, full paths to files
metadata_only : bool, default=False
    * Return before actually reading data if True
chans : list of str, default=None
    * List of desired channels (skip unneeded variables as needed).
    * Include all channels if None.
area_def : pyresample.AreaDefinition, default=None
    * Specify region 

Here we can see the call signature and docstring from the plugin.  
This should be enough to help you figure out what arguments to pass.

## Reading the ABI Test Data
  
For this test, we will use the data stored in the `GEOIPS_TESTDATA_DIR` directory.
  
We can access the current GeoIPS environment variables by calling:  
`geoips.filenames.base_paths.PATHS`  
which returns a dictionary with the environment variables.

In [4]:
from glob import glob
from geoips.filenames.base_paths import PATHS as GPATHS
GEOIPS_TESTDATA_DIR = GPATHS['GEOIPS_TESTDATA_DIR']
in_fpath = f"{GEOIPS_TESTDATA_DIR}/test_data_abi/data/goes16_20200918_1950/*"
in_fpaths = glob(in_fpath)

The `conus_sector_plugin` contains an `area_definition`.  
We want to use that `area_definition` when calling our reader.

In [17]:
conus_sector_area = conus_sector_plugin.area_definition
print(conus_sector_area)

Area ID: conus
Description: Continental United States
Projection: {'R': '6371228', 'lat_0': '39.5', 'lat_ts': '0', 'lon_0': '-98.35', 'no_defs': 'None', 'proj': 'eqc', 'type': 'crs', 'units': 'm', 'x_0': '0', 'y_0': '0'}
Number of columns: 2400
Number of rows: 1200
Area extent: (-3600000.0, -1800000.0, 3600000.0, 1800000.0)


First we need to read the abi data for channel 14 Brightness Temperature

In [6]:
xdict = abi_reader(fnames=in_fpaths, area_def=conus_sector_area, chans=["B14BT"])

# Since we passed an area_def for "conus", that is the name of the Xarray Object we want to call.
conus_xobj = xdict["conus"]

# Let's look at what conus_xobj contains within it:
conus_xobj

<xarray.Dataset> Size: 161MB
Dimensions:                  (dim_0: 1200, dim_1: 2400)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    B14BT                    (dim_0, dim_1) float64 23MB 266.3 266.5 ... 296.7
    latitude                 (dim_0, dim_1) float64 23MB nan nan ... 23.33 23.33
    longitude                (dim_0, dim_1) float64 23MB nan nan ... -65.99
    satellite_zenith_angle   (dim_0, dim_1) float64 23MB nan nan ... 29.26 29.27
    satellite_azimuth_angle  (dim_0, dim_1) float64 23MB nan nan ... 21.78 21.83
    solar_zenith_angle       (dim_0, dim_1) float64 23MB nan nan ... 54.19 54.21
    solar_azimuth_angle      (dim_0, dim_1) float64 23MB nan nan ... 4.439 4.439
Attributes: (12/13)
    file_metadata:                      {'B14': {'file_info': {'id': '0074e8b...
    start_datetime:                     2020-09-18 19:50:20.526449
    end_datetime:                       2020-09-18 19:59:51.325366
    source_name:                        abi
    data_provider:                      noaa
    source_file_names:                  ['OR_ABI-L1b-RadF-M6C14_G16_s20202621...
    ...                                 ...
    longitude_of_projection_origin:     -75.0
    perspective_point_height:           35786023.0
    platform_name:                      goes-16
    area_definition:                    Area ID: conus\nDescription: Continen...
    area_id:                            conus
    interpolation_radius_of_influence:  10000.0

## Applying the Interpolator
Now we will apply the interpolator. Let's look at the call signature:

In [7]:
interp_nearest?

Signature:     
interp_nearest(
    area_def,
    input_xarray,
    output_xarray,
    varlist,
    array_num=None,
    **kwargs,
)
Type:           InterpolatorsPlugin
String form:    <geoips.interfaces.base.InterpolatorsPlugin object at 0x7fdfce595390>
File:           ~/miniconda3/lib/python3.11/site-packages/geoips/interfaces/base.py
Docstring:      Geoips plugin for driving pyresample Nearest Neighbor interpolation.
Call docstring: Pyresample interp_kd_tree nearest neighbor GeoIPS plugin.

In [8]:
# Now let us apply the Nearest Neighbor interpolation to our data
output_dataset = interp_nearest(conus_sector_area,
                                conus_xobj,
                                None,
                                ["B14BT", "longitude", "latitude"])

# Let's look at our output_dataset
output_dataset

<xarray.Dataset> Size: 69MB
Dimensions:    (dim_0: 1200, dim_1: 2400)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    latitude   (dim_0, dim_1) float64 23MB nan nan nan nan ... 23.33 23.33 23.33
    longitude  (dim_0, dim_1) float64 23MB nan nan nan ... -66.04 -66.02 -65.99
    B14BT      (dim_0, dim_1) float64 23MB nan nan nan nan ... 295.1 296.5 296.7
Attributes:
    start_datetime:                     2020-09-18 19:50:20.526449
    end_datetime:                       2020-09-18 19:59:51.325366
    platform_name:                      goes-16
    source_name:                        abi
    data_provider:                      noaa
    source_file_names:                  ['OR_ABI-L1b-RadF-M6C14_G16_s20202621...
    interpolation_radius_of_influence:  10000.0
    area_definition:                    Area ID: conus\nDescription: Continen...
    longitude_of_projection_origin:     -75.0
    registered_dataset:                 True

In [9]:
import xarray as xr
channel_14_bt = output_dataset["B14BT"].data

# Now let us apply the single channel algorithm to our channel 14 Brightness Temperature data
algorithm_output = single_channel_algorithm([channel_14_bt],
                                            output_data_range=[-90.0, 30.0],
                                            input_units="Kelvin",
                                            output_units="celsius")

output_dataset["Infrared"] = xr.DataArray(algorithm_output)

# Let's look at this dataset now that we have created it
output_dataset

<xarray.Dataset> Size: 92MB
Dimensions:    (dim_0: 1200, dim_1: 2400)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    latitude   (dim_0, dim_1) float64 23MB nan nan nan nan ... 23.33 23.33 23.33
    longitude  (dim_0, dim_1) float64 23MB nan nan nan ... -66.04 -66.02 -65.99
    B14BT      (dim_0, dim_1) float64 23MB nan nan nan nan ... 295.1 296.5 296.7
    Infrared   (dim_0, dim_1) float64 23MB nan nan nan nan ... 21.95 23.36 23.56
Attributes:
    start_datetime:                     2020-09-18 19:50:20.526449
    end_datetime:                       2020-09-18 19:59:51.325366
    platform_name:                      goes-16
    source_name:                        abi
    data_provider:                      noaa
    source_file_names:                  ['OR_ABI-L1b-RadF-M6C14_G16_s20202621...
    interpolation_radius_of_influence:  10000.0
    area_definition:                    Area ID: conus\nDescription: Continen...
    longitude_of_projection_origin:     -75.0
    registered_dataset:                 True

## NetCDF Output Formatter

To start off, let's try putting our final data into a netcdf file

In [10]:
from datetime import datetime, timezone

ncdf_output_formatter = interfaces.output_formatters.get_plugin("netcdf_geoips")

timestamp = datetime.strftime(datetime.now(timezone.utc), "%Y%m%d%H%M%S")
GEOIPS_OUTDIRS = geoips.filenames.base_paths.PATHS['GEOIPS_OUTDIRS']
out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_xarray_test_{timestamp}.nc"

netcdf_output = ncdf_output_formatter(output_dataset,
                                        ["Infrared"],
                                        [out_fpath])
print(f"NetCDF output located at:  {netcdf_output[0]}")

NetCDF output located at:  /home/user02/geoips_outdirs/abi_infrared_xarray_test_20250731021006.nc


In [11]:
ncdf_output_formatter?

Signature:     
ncdf_output_formatter(
    xarray_obj,
    product_names,
    output_fnames,
    clobber=False,
)
Type:           OutputFormattersPlugin
String form:    <geoips.interfaces.base.OutputFormattersPlugin object at 0x7fdfb0eaab90>
File:           ~/miniconda3/lib/python3.11/site-packages/geoips/interfaces/base.py
Docstring:      Geoips style NetCDF output format.
Call docstring: Write GeoIPS style NetCDF to disk.

## Non-Annotated Imagery Output Formatter
We need a colormapper to tell matplotlib what colors we want to use

In [12]:
ir_colormapper = interfaces.colormappers.get_plugin("Infrared")

ir_color_dict = ir_colormapper()

In [13]:
img_clean_output_formatter = interfaces.output_formatters.get_plugin("imagery_clean")

out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_clean_test_{timestamp}.png"

png_clean_output = img_clean_output_formatter(conus_sector_area,
                                             output_dataset,
                                             "Infrared",
                                             [out_fpath],
                                             mpl_colors_info=ir_color_dict)

print(f"NetCDF output located at:  {png_clean_output[0]}")

NetCDF output located at:  /home/user02/geoips_outdirs/abi_infrared_clean_test_20250731021006.png


<div>
    <img src="../../geoips_outdirs/abi_infrared_clean_test_20250731005030.png" width="800" />
</div>
<!-- ![NetCDF Output Imagery](../../geoips_outdirs/abi_infrared_clean_test_20250731005030.png) -->

We need to tell matplotlib how we want the resulting plot to appear
using a feature_annotator and a gridline_annotator.
This time we will use the default versions of these plugins.

In [14]:
local_feature_annotator = interfaces.feature_annotators.get_plugin("default")

local_gridline_annotator = interfaces.gridline_annotators.get_plugin("default")

# Annotated Imagery
Now that we have all the setup completed, we can call our output formatter.  

In [21]:
# Let's start by loading the imagery_annotated plugin
img_ann_output_formatter = interfaces.output_formatters.get_plugin("imagery_annotated")

formatter_result_dict = {}
out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_annotated_test_{timestamp}.png"

# And then we call the plugin
png_annotated_output = img_ann_output_formatter(conus_sector_area,
                                           output_dataset,
                                           "Infrared",
                                           [out_fpath],
                                           mpl_colors_info=ir_color_dict,
                                           feature_annotator=local_feature_annotator,
                                           gridline_annotator=local_gridline_annotator,
                                           output_dict=formatter_result_dict)

# Let's print the path to the image on your disk
# print(png_annotated_output)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)>

![image](/home/coleman/geoips/outdirs/abi_infrared_annotated_test_20250718183819.png)

### Clean Up

Run the following segment to clean up the output files we created with this notebook:

In [ ]:
try:
    if netcdf_output:
        Path(netcdf_output[0]).unlink(missing_ok=True)
        print(f"Deleted {netcdf_output[0]}")
except NameError:
    None

try:
    if png_clean_output:
        Path(png_clean_output[0]).unlink(missing_ok=True)
        print(f"Deleted {png_clean_output[0]}")
except NameError:
    None

try:
    if png_annotated_output:
        Path(png_annotated_output[0]).unlink(missing_ok=True)
        print(f"Deleted {png_annotated_output[0]}")
except NameError:
    None